In [ ]:
import cv2
import numpy as np
import os

# Biến toàn cục để lưu vùng được chọn
drawing = False  # True nếu đang vẽ
ix, iy = -1, -1  # Toạ độ bắt đầu vẽ

def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing
    
    # Nếu nút chuột trái được nhấn
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
    
    # Nếu chuột di chuyển và đang vẽ
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy[:] = img.copy()
            cv2.rectangle(img_copy, (ix, iy), (x, y), (255, 255, 255), -1)  # Vẽ hình chữ nhật trắng
    
    # Nếu nút chuột trái được nhả ra
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img_copy, (ix, iy), (x, y), (255, 255, 255), -1)  # Hoàn tất vẽ

def create_mask(original_image_path, output_mask_path):
    global img, img_copy
    img = cv2.imread(original_image_path)
    img_copy = img.copy()
    
    # Tạo cửa sổ và gán hàm xử lý sự kiện
    cv2.namedWindow("Image")
    cv2.setMouseCallback("Image", draw_rectangle)

    while True:
        cv2.imshow("Image", img_copy)
        
        # Nhấn 'c' để hoàn tất
        if cv2.waitKey(1) & 0xFF == ord('c'):
            break

    # Lưu mask
    mask = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
    mask[mask > 0] = 255  # Chuyển các pixel khác 0 thành 255 (trắng)
    cv2.imwrite(output_mask_path, mask)
    
    cv2.destroyAllWindows()

# Đường dẫn đến ảnh gốc và thư mục lưu mặt nạ
original_image_path = './DenoisingImages/original/sample_image.jpg'  # Thay bằng ảnh gốc của bạn
output_mask_path = './DenoisingImages/masks/sample_mask.jpg'          # Thư mục lưu mặt nạ

create_mask(original_image_path, output_mask_path)


In [ ]:
import cv2
import numpy as np
import os

def inpaint_image(original_folder, mask_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for img_name in os.listdir(original_folder):
        img_path = os.path.join(original_folder, img_name)
        mask_path = os.path.join(mask_folder, img_name)

        # Đọc ảnh gốc
        img = cv2.imread(img_path)
        
        # Đọc ảnh mặt nạ
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Thực hiện inpainting
        inpainted_img = cv2.inpaint(img, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

        # Lưu ảnh đã inpaint
        cv2.imwrite(os.path.join(output_folder, img_name), inpainted_img)

# Đường dẫn đến thư mục chứa ảnh gốc và mặt nạ
original_folder = './DenoisingImages/original'  # Thay bằng thư mục chứa ảnh gốc của bạn
mask_folder = './DenoisingImages/masks'          # Thay bằng thư mục chứa mặt nạ (đen trắng) của bạn
output_folder = './DenoisingImages/inpainted'    # Thư mục lưu ảnh đã inpaint

inpaint_image(original_folder, mask_folder, output_folder)
